# RunnableLambda

`RunnableLambda` provides the ability to **execute user-defined functions**.

This allows developers to **define their own functions** and execute them using `RunnableLambda`.

For example, you can define and execute functions that perform tasks such as data preprocessing, calculations, or interactions with external APIs.

## How to Execute User-Defined Functions

**Caution**

User-defined functions can be wrapped and utilized with `RunnableLambda`. However, it is important to note that **the user-defined function can only accept one argument**.

If you want to implement a function that takes multiple arguments, you need to write a wrapper that accepts a single input and unpacks it into multiple arguments.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [1]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


def length_function(text):  # Function that returns the length of the text
    return len(text)


def _multiple_length_function(text1, text2):  # Function that multiplies the lengths of two texts
    return len(text1) * len(text2)


def multiple_length_function(  # Wrapper function that connects to a function that takes 2 arguments
    _dict,
):  # Function that multiplies the lengths of "text1" and "text2" from the dictionary
    return _multiple_length_function(_dict["text1"], _dict["text2"])


# Create prompt template
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# Initialize ChatOpenAI model
model = ChatOpenAI()

# Create chain by connecting prompt and model
chain1 = prompt | model

# Configure chain
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [2]:
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 equals 12.'